<a href="https://colab.research.google.com/github/Gooogr/Book_Ian_Pointer_PyTorch_for_DL/blob/master/Chapter%203%3A%20Convolutional%20Neural%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

from PIL import Image

import numpy as np
import cv2
import urllib.request

from google.colab.patches import cv2_imshow

In [2]:
! nvidia-smi

Wed Sep  2 20:56:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
BATCH_SIZE = 32
INPUT_SIZE = 64

**Prepare dataset**

In [4]:
! mkdir ch3
% cd ch2
! wget --quiet https://raw.githubusercontent.com/falloutdurham/beginners-pytorch-deep-learning/master/chapter2/download.py
! wget --quiet https://raw.githubusercontent.com/falloutdurham/beginners-pytorch-deep-learning/master/chapter2/images.csv

/content/ch2


In [ ]:
%%capture
!python download.py
% cd ../

# Takes ~ 20 min in google collab

In [ ]:
train_path = './ch3/train/'
val_path = './ch3/val/'
test_path = './ch3/test/'

In [ ]:
def check_image(path):
    '''
    Function will do a sanity check to make sure PIL can actually open the file.
    Passed to the is_valid_file parameter in the ImageFolder.
    '''
    try:
        im = Image.open(path)
        return True
    except:
        return False

Set image transformator

In [ ]:
img_transforms = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], #pre-compute constants
                         std=[0.229, 0.224, 0.225])
])

Set data generatos

In [ ]:
train_data = torchvision.datasets.ImageFolder(root=train_path, transform=img_transforms, 
                                              is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path, transform=img_transforms, 
                                            is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path, transform=img_transforms, 
                                             is_valid_file=check_image)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

**Prepare CNN (AlexNet)**

